In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler,LabelBinarizer
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier ,GradientBoostingClassifier
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

In [2]:
test_df = pd.read_csv('./datasets/mod_file/Test.csv')

In [3]:
# pd.set_option('display.max_columns',None)
test_df

,ID,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,17591,current,M,37,575,2795.450,1290.00,6141.050,676.50,9146.100,15479.00,4.0,10.0,4.0,47.0,56.0,6.0,125000.0,1.0,1.0,1.0,1.0,1.380000e+01,376600.50,1633.0,-5715.49023,27646.84,44341.00,16.0,36.0,24485.00,23512.67,41755.00,24.0,20.0,9750.00,30527.88,31271.00,3.0,12.0,25945.92,Y,1170.49
1,13541,current,M,33,394,29405.130,1640.00,1670.000,2463.92,10947.500,956.00,3.0,10.0,4.0,50.0,38.0,31.0,59000.0,1.0,1.0,1.0,1.0,1.018912e+05,470000.00,891.0,35346.00000,32247.25,21946.00,42.0,5.0,3375.00,15049.44,130299.00,43.0,40.0,16967.00,14342.83,16582.00,6.0,39.0,12214.00,Y,16447.45
2,13431,current,M,53,324,5985.200,6189.00,2696.090,766.00,716.000,4133.22,7.0,30.0,5.0,24.0,27.0,2.0,49999.0,1.0,1.0,1.0,1.0,5.337497e+04,674291.83,54101.0,2557.00000,63169.00,36616.50,41.0,6.0,14950.00,133659.00,40422.00,42.0,35.0,36398.00,32503.16,33539.54,5.0,9.0,13215.00,Y,2622.28
3,8687,current,M,33,370,2105.930,18225.00,34763.560,167.00,4260.270,20185.00,1.0,99.0,71.0,5.0,1.0,11.0,520000.0,1.0,1.0,1.0,1.0,9.118350e+03,470000.00,5905.0,5464.00000,39442.80,43173.38,46.0,1.0,235844.00,99686.03,48148.00,3.0,5.0,97825.00,93572.42,109429.75,21.0,0.0,72317.00,Y,340.79
4,14727,current,M,62,505,3269.000,3532.00,3158.400,2699.77,3373.480,5120.00,2.0,10.0,47.0,5.0,25.0,1.0,152000.0,1.0,1.0,1.0,1.0,1.285054e+07,101.24,130000.0,1332.00000,8937.00,15232.00,11.0,1.0,14495.00,190374.25,25926.00,70.0,75.0,27936.00,19011.50,41401.00,6.0,35.0,42344.00,Y,2812.20
5,14988,saving,M,30,454,11495.510,2701.00,3957.240,1395.00,36608.500,2296.00,2.0,23.0,35.0,21.0,62.0,13.0,276000.0,1.0,1.0,1.0,1.0,9.621853e+04,1155000.00,54101.0,-5998.43994,81568.39,9418.00,5.0,6.0,22599.00,46739.30,80813.00,2.0,8.0,27906.00,23978.50,32248.75,9.0,63.0,24986.00,Y,291.16
6,14859,current,M,33,796,3837.390,10809.00,126467.150,6204.00,3947.370,6288.00,5.0,11.0,50.0,21.0,17.0,3.0,39000.0,1.0,1.0,1.0,1.0,1.895040e+03,185097.60,15654.0,20324.00000,90521.16,109318.16,11.0,1.0,117591.00,16097.11,45735.00,2.0,20.0,11388.00,14789.48,33319.00,5.0,32.0,45228.00,Y,4786.40
7,16636,current,M,223,575,22928.500,6331.49,9974.840,2699.00,4578.000,704.00,531.0,1.0,74.0,7.0,3.0,12.0,393000.0,1.0,1.0,1.0,1.0,2.731650e+04,78050.00,13873.0,5730.00000,46865.63,41599.00,69.0,19.0,124720.00,30674.96,15929.00,3.0,20.0,69056.00,25376.75,81303.01,22.0,58.0,39930.00,Y,8186.99
8,7625,saving,M,37,623,2698.000,727.00,5264.500,5528.00,11310.500,627.44,26.0,9.0,9.0,1.0,2.0,17.0,381000.0,1.0,1.0,1.0,1.0,2.726772e+05,400000.00,3570.0,0.00000,36117.25,28601.00,77.0,33.0,57417.00,8299.00,36959.00,47.0,13.0,15700.00,22579.57,12317.00,6.0,12.0,63300.00,Y,54632.60
9,16492,current,M,34,123,6772.780,332.00,3343.560,8650.56,3672.500,4695.52,16.0,4.0,4.0,1.0,2.0,1.0,87000.0,1.0,1.0,1.0,1.0,2.151180e+04,100000.00,4458.0,-5942.00000,33792.55,98108.00,2.0,36.0,52341.00,18942.00,182908.00,47.0,11.0,77764.00,178334.88,111699.00,1.0,6.0,24559.00,Y,3796.29


In [4]:
test_df.shape

(5000, 43)

In [5]:
test_df.isna().sum().sort_values(ascending=False)/test_df.shape[0]*100

emi_active               0.0
dc_cons_jun              0.0
personal_loan_active     0.0
card_lim                 0.0
dc_count_jun             0.0
dc_count_may             0.0
dc_count_apr             0.0
cc_count_jun             0.0
cc_count_may             0.0
cc_count_apr             0.0
cc_cons_jun              0.0
personal_loan_closed     0.0
dc_cons_may              0.0
cc_cons_may              0.0
dc_cons_apr              0.0
cc_cons_apr              0.0
region_code              0.0
age                      0.0
gender                   0.0
account_type             0.0
vehicle_loan_active      0.0
vehicle_loan_closed      0.0
loan_enq                 0.0
credit_amount_may        0.0
max_credit_amount_jun    0.0
debit_count_jun          0.0
credit_count_jun         0.0
credit_amount_jun        0.0
debit_amount_jun         0.0
max_credit_amount_may    0.0
debit_count_may          0.0
credit_count_may         0.0
debit_amount_may         0.0
investment_1             0.0
max_credit_amo

In [6]:
len(test_df[test_df['age']>70]['age'])

172

In [7]:
def clean_test(dataset):
    dataset.loc[dataset['age']>70,'age']= dataset['age'].mode()[0]
    dataset.drop(['loan_enq','ID','personal_loan_closed','personal_loan_active','vehicle_loan_active','vehicle_loan_closed'],axis=1,inplace=True)
#     dataset.drop(,axis=1,inplace=True)
    
    return dataset
    

In [8]:
clean_test(test_df)

,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active
0,current,M,37,575,2795.450,1290.00,6141.050,676.50,9146.100,15479.00,4.0,10.0,4.0,47.0,56.0,6.0,125000.0,1.380000e+01,376600.50,1633.0,-5715.49023,27646.84,44341.00,16.0,36.0,24485.00,23512.67,41755.00,24.0,20.0,9750.00,30527.88,31271.00,3.0,12.0,25945.92,1170.49
1,current,M,33,394,29405.130,1640.00,1670.000,2463.92,10947.500,956.00,3.0,10.0,4.0,50.0,38.0,31.0,59000.0,1.018912e+05,470000.00,891.0,35346.00000,32247.25,21946.00,42.0,5.0,3375.00,15049.44,130299.00,43.0,40.0,16967.00,14342.83,16582.00,6.0,39.0,12214.00,16447.45
2,current,M,53,324,5985.200,6189.00,2696.090,766.00,716.000,4133.22,7.0,30.0,5.0,24.0,27.0,2.0,49999.0,5.337497e+04,674291.83,54101.0,2557.00000,63169.00,36616.50,41.0,6.0,14950.00,133659.00,40422.00,42.0,35.0,36398.00,32503.16,33539.54,5.0,9.0,13215.00,2622.28
3,current,M,33,370,2105.930,18225.00,34763.560,167.00,4260.270,20185.00,1.0,99.0,71.0,5.0,1.0,11.0,520000.0,9.118350e+03,470000.00,5905.0,5464.00000,39442.80,43173.38,46.0,1.0,235844.00,99686.03,48148.00,3.0,5.0,97825.00,93572.42,109429.75,21.0,0.0,72317.00,340.79
4,current,M,62,505,3269.000,3532.00,3158.400,2699.77,3373.480,5120.00,2.0,10.0,47.0,5.0,25.0,1.0,152000.0,1.285054e+07,101.24,130000.0,1332.00000,8937.00,15232.00,11.0,1.0,14495.00,190374.25,25926.00,70.0,75.0,27936.00,19011.50,41401.00,6.0,35.0,42344.00,2812.20
5,saving,M,30,454,11495.510,2701.00,3957.240,1395.00,36608.500,2296.00,2.0,23.0,35.0,21.0,62.0,13.0,276000.0,9.621853e+04,1155000.00,54101.0,-5998.43994,81568.39,9418.00,5.0,6.0,22599.00,46739.30,80813.00,2.0,8.0,27906.00,23978.50,32248.75,9.0,63.0,24986.00,291.16
6,current,M,33,796,3837.390,10809.00,126467.150,6204.00,3947.370,6288.00,5.0,11.0,50.0,21.0,17.0,3.0,39000.0,1.895040e+03,185097.60,15654.0,20324.00000,90521.16,109318.16,11.0,1.0,117591.00,16097.11,45735.00,2.0,20.0,11388.00,14789.48,33319.00,5.0,32.0,45228.00,4786.40
7,current,M,33,575,22928.500,6331.49,9974.840,2699.00,4578.000,704.00,531.0,1.0,74.0,7.0,3.0,12.0,393000.0,2.731650e+04,78050.00,13873.0,5730.00000,46865.63,41599.00,69.0,19.0,124720.00,30674.96,15929.00,3.0,20.0,69056.00,25376.75,81303.01,22.0,58.0,39930.00,8186.99
8,saving,M,37,623,2698.000,727.00,5264.500,5528.00,11310.500,627.44,26.0,9.0,9.0,1.0,2.0,17.0,381000.0,2.726772e+05,400000.00,3570.0,0.00000,36117.25,28601.00,77.0,33.0,57417.00,8299.00,36959.00,47.0,13.0,15700.00,22579.57,12317.00,6.0,12.0,63300.00,54632.60
9,current,M,34,123,6772.780,332.00,3343.560,8650.56,3672.500,4695.52,16.0,4.0,4.0,1.0,2.0,1.0,87000.0,2.151180e+04,100000.00,4458.0,-5942.00000,33792.55,98108.00,2.0,36.0,52341.00,18942.00,182908.00,47.0,11.0,77764.00,178334.88,111699.00,1.0,6.0,24559.00,3796.29


In [15]:
# Function to treat outliers 
def treat_outliers(dataframe):
    cols = list(dataframe)
    for col in cols:
        if col in dataframe.select_dtypes(include=np.number).columns:
            dataframe[col] = winsorize(dataframe[col], limits=[0.05, 0.1],inclusive=(True, True))
    
    return dataframe    


treat_outliers(test_df)

# Checking for outliers after applying winsorization
# detect_outliers(test_df)

,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active
0,current,M,37,575,2795.450,1290.00,6141.050,676.50,9146.100,15479.00,4.0,10.0,4.0,47.0,44.0,6.0,125000.0,168.7500,376600.50,1633.0,-5715.49023,27646.84,44341.00,16.0,35.0,24485.00,23512.67,41755.00,24.0,20.0,9750.00,30527.88,31271.00,3.0,12.0,25945.92,1170.49
1,current,M,33,394,29405.130,1640.00,1670.000,2463.92,10947.500,956.00,3.0,10.0,4.0,50.0,38.0,31.0,59000.0,101891.2500,470000.00,891.0,12299.00000,32247.25,21946.00,42.0,5.0,10100.00,15049.44,130299.00,43.0,40.0,16967.00,14342.83,16582.00,6.0,39.0,12214.00,16447.45
2,current,M,53,324,5985.200,6189.00,2696.090,766.00,813.000,4133.22,7.0,30.0,5.0,24.0,27.0,2.0,49999.0,53374.9700,470000.00,54101.0,2557.00000,63169.00,36616.50,41.0,6.0,14950.00,133659.00,40422.00,42.0,35.0,36398.00,32503.16,33539.54,5.0,9.0,13215.00,2622.28
3,current,M,33,370,2105.930,18225.00,34763.560,411.00,4260.270,15620.69,1.0,83.0,71.0,5.0,1.0,11.0,520000.0,9118.3500,470000.00,5905.0,5464.00000,39442.80,43173.38,46.0,1.0,94812.00,99686.03,48148.00,3.0,5.0,97825.00,93572.42,109429.75,21.0,1.0,72317.00,340.79
4,current,M,62,505,3269.000,3532.00,3158.400,2699.77,3373.480,5120.00,2.0,10.0,47.0,5.0,25.0,1.0,152000.0,720551.4000,2625.16,126001.0,1332.00000,8937.00,15232.00,11.0,1.0,14495.00,134968.36,25926.00,43.0,75.0,27936.00,19011.50,41401.00,6.0,35.0,42344.00,2812.20
5,saving,M,30,454,11495.510,2701.00,3957.240,1395.00,26166.960,2296.00,2.0,23.0,35.0,21.0,44.0,13.0,276000.0,96218.5300,470000.00,54101.0,-5998.43994,81568.39,9418.00,5.0,6.0,22599.00,46739.30,80813.00,2.0,8.0,27906.00,23978.50,32248.75,9.0,63.0,24986.00,291.16
6,current,M,33,796,3837.390,10809.00,35562.320,6204.00,3947.370,6288.00,5.0,11.0,50.0,21.0,17.0,3.0,39000.0,1895.0400,185097.60,15654.0,12299.00000,90521.16,109318.16,11.0,1.0,94812.00,16097.11,45735.00,2.0,20.0,11388.00,14789.48,33319.00,5.0,32.0,45228.00,4786.40
7,current,M,33,575,22928.500,6331.49,9974.840,2699.00,4578.000,704.00,67.0,2.0,74.0,7.0,3.0,12.0,393000.0,27316.5000,78050.00,13873.0,5730.00000,46865.63,41599.00,69.0,19.0,94812.00,30674.96,15929.00,3.0,20.0,69056.00,25376.75,81303.01,22.0,58.0,39930.00,8186.99
8,saving,M,37,623,2698.000,727.00,5264.500,5528.00,11310.500,627.44,26.0,9.0,9.0,1.0,2.0,17.0,381000.0,272677.2500,400000.00,3570.0,0.00000,36117.25,28601.00,77.0,33.0,57417.00,8299.00,36959.00,43.0,13.0,15700.00,22579.57,12317.00,6.0,12.0,63300.00,19363.91
9,current,M,34,249,6772.780,383.30,3343.560,8650.56,3672.500,4695.52,16.0,4.0,4.0,1.0,2.0,1.0,87000.0,21511.8000,100000.00,4458.0,-5942.00000,33792.55,98108.00,3.0,35.0,52341.00,18942.00,171192.00,43.0,11.0,77764.00,154679.36,111699.00,2.0,6.0,24559.00,3796.29


In [24]:
# test_df

In [16]:
le = LabelEncoder()
# Function that auto encodes any dataframe column of type category or object.
def dummyEncode(dataset):
        
        columnsToEncode = list(dataset.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                dataset[feature] = le.fit_transform(dataset[feature])
            except:
                print('Error encoding '+feature)
        return dataset


In [17]:
# test_df

In [18]:
encode_test = dummyEncode(test_df)

In [19]:
encode_test

,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active
0,0,1,37,575,2795.450,1290.00,6141.050,676.50,9146.100,15479.00,4.0,10.0,4.0,47.0,44.0,6.0,125000.0,168.7500,376600.50,1633.0,-5715.49023,27646.84,44341.00,16.0,35.0,24485.00,23512.67,41755.00,24.0,20.0,9750.00,30527.88,31271.00,3.0,12.0,25945.92,1170.49
1,0,1,33,394,29405.130,1640.00,1670.000,2463.92,10947.500,956.00,3.0,10.0,4.0,50.0,38.0,31.0,59000.0,101891.2500,470000.00,891.0,12299.00000,32247.25,21946.00,42.0,5.0,10100.00,15049.44,130299.00,43.0,40.0,16967.00,14342.83,16582.00,6.0,39.0,12214.00,16447.45
2,0,1,53,324,5985.200,6189.00,2696.090,766.00,813.000,4133.22,7.0,30.0,5.0,24.0,27.0,2.0,49999.0,53374.9700,470000.00,54101.0,2557.00000,63169.00,36616.50,41.0,6.0,14950.00,133659.00,40422.00,42.0,35.0,36398.00,32503.16,33539.54,5.0,9.0,13215.00,2622.28
3,0,1,33,370,2105.930,18225.00,34763.560,411.00,4260.270,15620.69,1.0,83.0,71.0,5.0,1.0,11.0,520000.0,9118.3500,470000.00,5905.0,5464.00000,39442.80,43173.38,46.0,1.0,94812.00,99686.03,48148.00,3.0,5.0,97825.00,93572.42,109429.75,21.0,1.0,72317.00,340.79
4,0,1,62,505,3269.000,3532.00,3158.400,2699.77,3373.480,5120.00,2.0,10.0,47.0,5.0,25.0,1.0,152000.0,720551.4000,2625.16,126001.0,1332.00000,8937.00,15232.00,11.0,1.0,14495.00,134968.36,25926.00,43.0,75.0,27936.00,19011.50,41401.00,6.0,35.0,42344.00,2812.20
5,1,1,30,454,11495.510,2701.00,3957.240,1395.00,26166.960,2296.00,2.0,23.0,35.0,21.0,44.0,13.0,276000.0,96218.5300,470000.00,54101.0,-5998.43994,81568.39,9418.00,5.0,6.0,22599.00,46739.30,80813.00,2.0,8.0,27906.00,23978.50,32248.75,9.0,63.0,24986.00,291.16
6,0,1,33,796,3837.390,10809.00,35562.320,6204.00,3947.370,6288.00,5.0,11.0,50.0,21.0,17.0,3.0,39000.0,1895.0400,185097.60,15654.0,12299.00000,90521.16,109318.16,11.0,1.0,94812.00,16097.11,45735.00,2.0,20.0,11388.00,14789.48,33319.00,5.0,32.0,45228.00,4786.40
7,0,1,33,575,22928.500,6331.49,9974.840,2699.00,4578.000,704.00,67.0,2.0,74.0,7.0,3.0,12.0,393000.0,27316.5000,78050.00,13873.0,5730.00000,46865.63,41599.00,69.0,19.0,94812.00,30674.96,15929.00,3.0,20.0,69056.00,25376.75,81303.01,22.0,58.0,39930.00,8186.99
8,1,1,37,623,2698.000,727.00,5264.500,5528.00,11310.500,627.44,26.0,9.0,9.0,1.0,2.0,17.0,381000.0,272677.2500,400000.00,3570.0,0.00000,36117.25,28601.00,77.0,33.0,57417.00,8299.00,36959.00,43.0,13.0,15700.00,22579.57,12317.00,6.0,12.0,63300.00,19363.91
9,0,1,34,249,6772.780,383.30,3343.560,8650.56,3672.500,4695.52,16.0,4.0,4.0,1.0,2.0,1.0,87000.0,21511.8000,100000.00,4458.0,-5942.00000,33792.55,98108.00,3.0,35.0,52341.00,18942.00,171192.00,43.0,11.0,77764.00,154679.36,111699.00,2.0,6.0,24559.00,3796.29


In [20]:
train = pd.read_csv('./datasets/clean/clean_train.csv')

In [21]:
X= train.iloc[:,:-1]
y =train['cc_cons']
linreg = LinearRegression()
linreg.fit(X,y)
y_pred = linreg.predict(encode_test)

In [22]:
y_pred

array([42231.21974803, 47683.52775513, 44721.12955229, ...,
       42063.51557898, 43973.43568613, 44866.29574068])

In [23]:
def rmsle(actual_column, predicted_column):
    sum=0.0
    for x,y in zip(actual_column,predicted_column):
        if x<0 or y<0: #check for negative values. 
            continue
        p = np.log(y+1)
        r = np.log(x+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted_column))**0.5

In [24]:
rmsle(y,y_pred)

1.7819524621096876